# Non Linear Programing (NLP)

Same structure as Linear Programing (LP) but with the difference that:

$ minf(x)$

subject to

$ h(x) = 0 $

$ g(x) <= 0 $

- **The main difference is that f,h or g are non linear**
- x is all continuous variable

## Example 1

consider the nonlinear regression problem using the data x and y, where *y* is the dependent variable and *x* is the independent variable

- 6 observations/data points are avilable fot this regression model.

- The required model structure is $y= b_1 + b_2e^{b_3x}$

|y          |x         |
|-----------|----------|
|127        |-5        |
|151        |-3        |
|379        |-1        |
|421        |5         |
|460        |3         |
|426        |1         |


**OBJ: Find the optimal values of regression coefficients $b_1,b_2, b_3$**

**Example Linear Regresion approach**

- Solve using 'least squares' regression approach gives the optimization model the following function:

$$min \sum_{i=1}(y_i-(a_1+a_2x_i))^2 \forall i \in m$$

**m** are all the data points

## Example 1 - proposed nonlinear regression approach

- Solve using 'least squares' regression approach gives:

$$min \sum_{i=1}(y_i-(b_1+b_2e^{b_3x}))^2 \forall i \in m$$

- Given bounds on the coefficients:
    - $-5 \leq b_3 \leq 5$

#### Important: On non linear optimization the solution dependes HEAVILY on the starting point or initial solution, ITS IMPORTANT TO PROVE A GOOD FEASIBLE SOLUTION TO THE SOLVER

In [1]:
import pyomo.environ as pyomo
import numpy as np

# Model definition
model_linear = pyomo.ConcreteModel();
model_nonlinear = pyomo.ConcreteModel();

# Set declaration, Defina a sequence of nuemeric elements
model_linear.m = pyomo.RangeSet(6);
model_nonlinear.m = pyomo.RangeSet(6);

# Variable definition FOR THE LINEAR MODEL
model_linear.a1 = pyomo.Var(domain=pyomo.Reals);
model_linear.a2 = pyomo.Var(domain=pyomo.Reals);

# Variable definition FOR THE NON LINEAR MODEL
model_nonlinear.b1 = pyomo.Var(domain=pyomo.Reals, initialize = 500);
model_nonlinear.b2 = pyomo.Var(domain=pyomo.Reals, initialize = -150);
model_nonlinear.b3 = pyomo.Var(domain=pyomo.Reals, initialize = -0.2,bounds = (-5,5));

# Using inicialize we start the variables on the predefined starting points on the optimization
model_linear.datapoints_y = pyomo.Param(model_linear.m,initialize = {1:127,2:151,3:379,4:421,5:460,6:426});
model_linear.datapoints_x = pyomo.Param(model_linear.m,initialize = {1:-5,2:-3,3:-1,4:5,5:3,6:1});

model_nonlinear.datapoints_y = pyomo.Param(model_nonlinear.m,initialize = {1:127,2:151,3:379,4:421,5:460,6:426});
model_nonlinear.datapoints_x = pyomo.Param(model_nonlinear.m,initialize = {1:-5,2:-3,3:-1,4:5,5:3,6:1});

# Objective functions
model_linear.obj = pyomo.Objective(expr = sum((model_linear.datapoints_y[m] - 
                                               (model_linear.a1 + model_linear.a2*model_linear.datapoints_x[m]))**2 
                                              for m in model_linear.m),sense = pyomo.minimize);
model_nonlinear.obj = pyomo.Objective(expr = sum((model_nonlinear.datapoints_y[m] - 
                                               (model_nonlinear.b1 + model_nonlinear.b2*pyomo.exp(model_nonlinear.b3*model_nonlinear.datapoints_x[m])))**2 
                                              for m in model_nonlinear.m),sense = pyomo.minimize);

# Solver options

results = pyomo.SolverFactory('ipopt').solve(model_linear);

results.write()
print("\n RESULTS \n");
print("Squared deviation for linear regression model = ",model_linear.obj());
print("Coefficient 1 for linear regression (a1) = ",model_linear.a1());
print("Coefficient 2 for linear regression (a2) = ",model_linear.a2());

results = pyomo.SolverFactory('ipopt',executable = '/content/ipopt').solve(model_nonlinear);

results.write()
print("\n RESULTS \n");
print("Squared deviation for nonlinear regression model = ",model_nonlinear.obj());
print("Coefficient 1 for nonlinear regression (b1) = ",model_nonlinear.b1());
print("Coefficient 2 for nonlinear regression (b2) = ",model_nonlinear.b2());
print("Coefficient 3 for nonlinear regression (b3) = ",model_nonlinear.b3());

    ipopt


ApplicationError: No executable found for solver 'ipopt'